In [ ]:
%load_ext autoreload
%autoreload 2

### **CLIP-I, CLIP-T, DINO**

- **CLIP-I** - average pairwise cosine similarity between CLIP embeddings of generated and real images
- **CLIP-T** - average cosine similarity between prompt and CLIP embeddings of generated images
- **DINO** - average pairwise cosine similarity between the Vit-S/16 DINO embeddings of generated and real images

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from torch.nn import functional as F
from transformers import AutoModel, AutoProcessor, AutoTokenizer

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [ ]:
img_path_1 = '/net/tscratch/people/plgkzaleska/ziplora-analysis/data/dreambooth/dataset/rc_car/01.jpg'
img_path_2 = '/net/tscratch/people/plgkzaleska/ziplora-analysis/data/dreambooth/dataset/rc_car/02.jpg'
img_path_3 = '/net/tscratch/people/plgkzaleska/ziplora-analysis/data/dreambooth/dataset/rc_car/03.jpg'

In [ ]:
image1 = Image.open(img_path_1)
image2 = Image.open(img_path_2)
image3 = Image.open(img_path_3)

In [ ]:
# display images in figure
fig, ax = plt.subplots(1, 3, figsize=(15, 15))
ax[0].imshow(image1)
ax[1].imshow(image2)
ax[2].imshow(image3)
# hide the axes
for ax in ax:
    ax.axis('off')
plt.show()

#### **CLIP-I**

In [ ]:
clip_processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = AutoModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

In [ ]:
inputs1 = clip_processor(images=image1, return_tensors="pt").to(device)
image_features1 = clip_model.get_image_features(**inputs1)

inputs2 = clip_processor(images=image2, return_tensors="pt").to(device)
image_features2 = clip_model.get_image_features(**inputs2)

In [ ]:
sim = F.cosine_similarity(image_features1[0], image_features2[0], dim=0)
print(f"Similarity: {sim.item():.3f}")

#### **CLIP-T**

In [ ]:
clip_tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
input_text = "a photo of a toy car"
text = clip_tokenizer(input_text, return_tensors="pt", padding=True).to(device)

In [ ]:
input_img = clip_processor(images=image1, return_tensors="pt").to(device)

In [ ]:
img_embeddings = clip_model.get_image_features(**input_img)
text_features = clip_model.get_text_features(**text)

In [ ]:
sim = F.cosine_similarity(img_embeddings[0], text_features[0], dim=0)
print(f"Similarity: {sim.item():.3f}")

#### **DINO**

In [ ]:
dino_model = AutoModel.from_pretrained("facebook/dino-vits16", add_pooling_layer=False).to(device)

In [ ]:
T = transforms.Compose([
    transforms.Resize(256, interpolation=3),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [ ]:
# get DINO features
inputs1 = T(image1).unsqueeze(0).to(device)
image_features1 = dino_model(inputs1).last_hidden_state

inputs2 = T(image2).unsqueeze(0).to(device)
image_features2 = dino_model(inputs2).last_hidden_state

In [ ]:
sim = F.cosine_similarity(image_features1[0, 0], image_features2[0, 0], dim=0)
print('Similarity:', sim.item())